In [1]:
import os 
import sys
import matplotlib.pyplot as plt 
import numpy as np 

import torch
import torch.nn as nn 
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

sys.path.append(r"..\Dataset\Segmentation")
sys.path.append(r"..\Backbone\Dense")

from Tiramisu import *
from VOCseg import *


In [2]:
model = Tiramisu103( num_classes=12, init_weight = False,)
#model.cuda()
model = nn.DataParallel(model)
model_chẹkpoints = torch.load(r'..\Models\Tiramisu\tiramisu103.pth', map_location="cuda")
print('loading Model')
model.load_state_dict(model_chẹkpoints)

loading Model


<All keys matched successfully>

In [3]:
model_one = Tiramisu103(num_classes=2, init_weight=True)
model_one = nn.DataParallel(model_one)


In [ ]:
# ==> for class 
def load_pretrained_weight(self, path_pretrained = r'..\Models\Tiramisu\tiramisu103.pth', watching = True):
    # load pretrained state dict
    pretrained_dict = torch.load(pretrained)
    # get state dict
    model_dict = self.state_dict()
    # 1. filter out unnecessary keys
    tmp_pretrained_dict = {}
    for k, v in list(pretrained_dict.items())[:-2]:
      if k in model_dict:
        if watching:
            print(f'match {k}')
        tmp_pretrained_dict.update({k: v})
      else:
        if watching:
            print(f'unmatch {k}')
    # 2. overwrite entries in the existing state dict
    model_dict.update(tmp_pretrained_dict)
    # 3. load the new state dict
    self.load_state_dict(model_dict)


In [22]:
def load_weights(model, pretrained = r'..\Models\Tiramisu\tiramisu103.pth', n_classes= 2):
    pretrained_dict = torch.load(pretrained)
    model_dict = model.state_dict()

    # 1. filter out unnecessary keys
    #pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    tmp_pretrained_dict = {}
    for k, v in list(pretrained_dict.items())[:-2]:
      
      if k in model_dict:
        print(f'match {k}')
        tmp_pretrained_dict.update({k: v})
      else:
        print(f'unmatch {k}')
    # 2. overwrite entries in the existing state dict
    model_dict.update(tmp_pretrained_dict)

    # 3. load the new state dict
    model.load_state_dict(model_dict)

    return model


In [24]:
model_one_2 = load_weights(model = model_one)


match module.model.0.weight
match module.model.0.bias
match module.model.1.dense_block.module.0.module.BN.weight
match module.model.1.dense_block.module.0.module.BN.bias
match module.model.1.dense_block.module.0.module.BN.running_mean
match module.model.1.dense_block.module.0.module.BN.running_var
match module.model.1.dense_block.module.0.module.BN.num_batches_tracked
match module.model.1.dense_block.module.0.module.CONV.weight
match module.model.1.dense_block.module.0.module.CONV.bias
match module.model.1.dense_block.module.1.module.BN.weight
match module.model.1.dense_block.module.1.module.BN.bias
match module.model.1.dense_block.module.1.module.BN.running_mean
match module.model.1.dense_block.module.1.module.BN.running_var
match module.model.1.dense_block.module.1.module.BN.num_batches_tracked
match module.model.1.dense_block.module.1.module.CONV.weight
match module.model.1.dense_block.module.1.module.CONV.bias
match module.model.1.dense_block.module.2.module.BN.weight
match module.

In [25]:
x = model.module.model[5].dense_block.module[0].module[-2].weight
y = model_one.module.model[5].dense_block.module[0].module[-2].weight
print(x.ne(y).sum())

tensor(0, device='cuda:0')


In [19]:
model.module.LastLayer

Sequential(
  (point_wise): Conv2d(256, 12, kernel_size=(1, 1), stride=(1, 1))
  (softmax): LogSoftmax()
)

In [21]:
model_one_2.module.LastLayer

Sequential(
  (point_wise): Conv2d(256, 1, kernel_size=(1, 1), stride=(1, 1))
  (softmax): LogSoftmax()
)

In [16]:
path = r'..\Models\Tiramisu\tiramisu103_2class.pth'
model_one_2.save_dict(path)

AttributeError: 'DataParallel' object has no attribute 'save_dict'

In [17]:
torch.save(model_one_2.state_dict(), path)

In [26]:
model_one

DataParallel(
  (module): Tiramisu(
    (model): ModuleList(
      (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): DownStep(
        (dense_block): DenseBlock(
          (module): ModuleList(
            (0): DenseLayer(
              (module): Sequential(
                (BN): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (ReLU): ReLU(inplace=True)
                (CONV): Conv2d(48, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                (Drop): Dropout2d(p=0.2, inplace=False)
              )
            )
            (1): DenseLayer(
              (module): Sequential(
                (BN): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (ReLU): ReLU(inplace=True)
                (CONV): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                (Drop): Dropout2d(p=0.2, inplace=False)
              )
            )
   